In [1]:
!pip install -q pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.4 MB/s eta 0:00:00


In [2]:
!wget -N -q "https://github.com/thomasfork/ipopt_linux/raw/main/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [3]:
import numpy as np
from pyomo.environ import *

In [4]:
def rastrigin(model):
    delta = 0.
    for i in model.x:
      delta = delta + model.x[i]**2 - model.A[1] * cos(2 * np.pi * model.x[i])
    return model.A[1] * model.N_unknowns[1] + delta

In [7]:
# --- Create a model
model         = ConcreteModel()

N_unknowns          = 10
model.N_unknowns    = Param(RangeSet(1, 1), initialize = N_unknowns, within = Integers)
model.i             = RangeSet(1, N_unknowns)
model.x             = Var(model.i, within = Reals)
A                   = 10.0
model.A             = Param(RangeSet(1, 1), initialize = A, within = Reals)
for i in model.i:
  model.x[i].value = .3
  model.x[i].lower = -5.12
  model.x[i].upper =  5.12

# --- Declare objective function
model.objfun    = Objective(rule = rastrigin, sense = minimize)

model.pprint()

1 RangeSet Declarations
    i : Dimen=1, Size=10, Bounds=(1, 10)
        Key  : Finite : Members
        None :   True :  [1:10]

2 Param Declarations
    A : Size=1, Index=[1], Domain=Reals, Default=None, Mutable=False
        Key : Value
          1 :  10.0
    N_unknowns : Size=1, Index=[1], Domain=Integers, Default=None, Mutable=False
        Key : Value
          1 :    10

1 Var Declarations
    x : Size=10, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 : -5.12 :   0.3 :  5.12 : False : False :  Reals
          2 : -5.12 :   0.3 :  5.12 : False : False :  Reals
          3 : -5.12 :   0.3 :  5.12 : False : False :  Reals
          4 : -5.12 :   0.3 :  5.12 : False : False :  Reals
          5 : -5.12 :   0.3 :  5.12 : False : False :  Reals
          6 : -5.12 :   0.3 :  5.12 : False : False :  Reals
          7 : -5.12 :   0.3 :  5.12 : False : False :  Reals
          8 : -5.12 :   0.3 :  5.12 : False : False :  Reals
          9 : -5.12 :   0

In [8]:
SolverFactory('ipopt', executable='/content/ipopt').solve(model).write()
#SolverFactory('multistart').solve(model, strategy = 'rand_guess_and_bound', solver = 'ipopt', iterations = 50).write()

# display solution
print('\nObjective function = ', model.objfun)

print('\nDecision Variables')
for i in model.x:
  print(model.x[i]())

print(model.A[1])
print(model.N_unknowns[1])

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 10
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.012269020080566406
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Obje